In [ ]:
# import utilities and plotting
from sys import path
path.append('../lib')

%matplotlib notebook
from argparse import Namespace
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import display
# make figures higher resolution (larger)
plt.rcParams['figure.dpi'] = 150
# A decent backend for animations
plt.rcParams["animation.html"] = "jshtml"
# let 'em get big
plt.rcParams["animation.embed_limit"] = 100  # MB

%load_ext autoreload
%autoreload 2

# import our code that we need
import fixed_center_of_mass_exact as fcm
#import fixed_center_of_mass_exact as fsm
from davidson import solve_davidson
from constants import *
from hamiltonian import solve_BO_surfaces

# don't use too many cores
from pyscf import lib as pyscflib
pyscflib.num_threads(16)

In [ ]:
# set up the calculation
args = Namespace(
    M_1 = 1.0,
    M_2 = 1.0,
    g_1 = 1.0,
    g_2 = 1.0,
    NR = 150,
    Nr = 400,
    Ng = 200,
    J = 0
    #extent = (2,4,0,5)  # defaults are (2,4,0,5), but they should be selected better
)

# build the terms in the Hamiltonian
TR, Tr, Tg, Vgrid, Hx, (R,P), (r,p), (g, pg) = fcm.build_terms(args)


levels = np.linspace(np.min(Vgrid), np.min(Vgrid) + 0.15, 16) # 0.15 a.u. ~4 eV range

In [ ]:
# plot many polar slices in R (what the electron sees)

def init(fig, ax):  # sets up colorbar
    ax.contour(*np.meshgrid(g,r, indexing='ij'), Vgrid[0,:,:].T, levels=levels)
    cs = ax.contourf(*np.meshgrid(g,r, indexing='ij'), Vgrid[0,:,:].T, levels=levels) # dummy contour with filled patches
    cbar = fig.colorbar(cs, orientation='horizontal', fraction=0.05)
    cs.remove()
    cbar.set_label("E / a.u.")

def animate(t):
    ax.cla()
    ax.contour(*np.meshgrid(g,r, indexing='ij'), Vgrid[t,:,:].T, levels=levels)
    ax.text(np.pi/2,6,f"R={R[t]:0.03}a₀", ha='center')

    ax.grid(axis='x')
    locs=ax.get_xticks()
    labels = [f'{th/np.pi:.03}π' for th in locs]
    ax.set_xticks(locs, labels)
    [limit]=ax.get_yticks()[-1:]
    ax.set_yticks([limit], [f"r={limit}a₀"])


fig, ax = plt.subplots(subplot_kw=dict(projection='polar'))
init(fig, ax)

ani=matplotlib.animation.FuncAnimation(fig, animate, frames=args.NR)
display(ani)

In [ ]:
# plot many slices, but this time just the radial components
fig, ax = plt.subplots()
def animate(t):
    ax.cla()
    #for c in ax.collections: c.remove
    ax.contour(*np.meshgrid(R, r, indexing='ij'), Vgrid[:,:,t], levels=levels)
    ax.set_xlabel("R / a.u.")
    ax.set_ylabel("r / a.u.")
    ax.text(4,8,f"γ={g[t]/np.pi:0.03}π")

ani=matplotlib.animation.FuncAnimation(fig, animate, frames=args.Ng)
display(ani)

In [ ]:
from hamiltonian import KE, KE_FFT
Nx=5
x=np.linspace(0,1,Nx)
dx=x[1]-x[0]
px  = np.fft.fftshift(np.fft.fftfreq(Nx, dx)) * 2 * np.pi
T_fft = np.real(KE_FFT(Nx, px, x, 1))
T_stn = KE(Nx, dx, 1, stencil_size=3)
print(T_fft)
print(T_stn)
print(T_stn/T_fft)

In [ ]:
levels = np.linspace(np.min(Vgrid), 0.08, 15)
fig, ax = plt.subplots()
ax.contour(*np.meshgrid(R, r, indexing='ij'), Vgrid, levels=levels)
ax.set_xlabel("R / a.u.")
ax.set_ylabel("r / a.u.")
ax.set_title("Potential energy surface")

In [ ]:
# plot the first 5 BO surfaces
fix, ax = plt.subplots()
colors=['r','b','g','k','m']
for i in range(5):
    ax.plot(R, surfs[i], color=colors[i])
    _, psi_bo = np.linalg.eigh(TR+np.diag(surfs[i]))
    psi_bo2 = psi_bo[:,0]**2
    ax.plot(R, psi_bo2/np.max(psi_bo2)*0.02+np.min(surfs[i]), color=colors[i], label=f'state {i}')
ax.legend(loc='best')
ax.set_xlabel("R / a.u.")
ax.set_ylabel("E / a.u.")
plt.ylim(-0.06,0.03)
ax.set_title("Born-Oppenhimer States")

In [ ]:
# plot the guesses
from davidson import build_preconditioner
_, guesses = build_preconditioner(TR, Tr+Tmp, Vgrid)
N=len(guesses)

levels = np.linspace(np.min(Vgrid), 0.08, 10)
fix, axs = plt.subplots(1,N, sharey=True)
for i, ax in enumerate(axs):
    ax.contour(*np.meshgrid(R, r, indexing='ij'), Vgrid, levels=levels)
    psi = guesses[i].reshape(args.NR,args.Nr)
    limit = np.max(np.abs(psi))
    ax.imshow(psi.T, extent=(args.extent * ANGSTROM_TO_BOHR), origin='lower', cmap='seismic', vmin=-limit, vmax=limit)
fig.suptitle("Trial wavefunctions")

In [ ]:
# solve the full system (if this is the first time running, get rid of the guess)
conv, e_approx, evecs = solve_davidson(TR, Tr + Tmp, Vgrid, num_state=10)

In [ ]:
# plot the potential and the first excited wavefunction (with phase information)
levels = np.linspace(np.min(Vgrid), 0.08, 10)
fix, axs = plt.subplots(3,3, sharey='row', sharex='col', constrained_layout=True)
for i, ax in enumerate(axs.flatten()):
    ax.contour(*np.meshgrid(R, r, indexing='ij'), Vgrid, levels=levels)
    psi = evecs[i].reshape(args.NR,args.Nr)
    limit = np.max(np.abs(psi))
    ax.imshow(psi.T, extent=(args.extent * ANGSTROM_TO_BOHR), origin='lower', cmap='seismic', vmin=-limit, vmax=limit)
fig.suptitle("Solution wavefunctions")